# Programação Orientada a Objetos - Aula 6

# Atributos estáticos

- Atributo que pertence à classe, e não ao objeto instanciado daquela classe. 

Chamado também de:
- `atributo de classe` ou `variável de classe`

- Declarado diretamente na classe

```python
class Teste():
    atributo_estatico = 'nome'
```

- Acessível independente de instâncias da classe

Sintaxe para uso:

```python
nome_da_classe.nome_do_atributo
```

Exemplo de uso: Controle dos CPFs já cadastrados

Como controlar efetivamente os CPFs?

Opção 1:
- Validar os CPFs no nosso programa principal

Opção 2:
- Validar o CPF no construtor da classe Titular:

Problema!!!

Uso do nome da classe. Em casos de refatoração, como fazer?

```python
nome_da_classe.nome_do_atributo
```

# Métodos de classe

- Método que pertence à classe, e recebe a classe como parâmetro: `cls`

Declarado da seguinte forma:

```python
@classmethod
def metodo_de_classe(cls, parametros):
```

> Importante: nos métodos de classe passamos a referência à classe usando normalmente o `cls`

&nbsp;

Sintaxe para uso:

```python
instancia_do_objeto.nome_do_metodo_da_classe()
```

Exemplo de uso: Controle dos CPFs já cadastrados

Outro caso de uso: Factory

Nesse exemplo, sobrecarga de construtores.

# Métodos estáticos

- Método que não deveria afetar o estdo do objeto e nem da classe.
- Poderia estar fora da classe, mas por conveniência (já que normalmente envolve "assuntos" da classe) fica dentro dela.

Declarado da seguinte forma: 

```python
@staticmethod
def metodo_estatico(parametros):
```

> Importante: nos métodos estáticos, não passamos a referência ao objeto `self` e nem a referência da classe `cls`


&nbsp;

Sintaxe para uso:

```python
nome_da_classe.nome_do_metodo(parametros)
```

Exemplo de uso: Validação de CPF